In [9]:
import pandas as pd 
import numpy as np
import os
from functools import reduce

In [2]:
print(os.getcwd())

c:\Users\sydwi\Desktop\QSML\data\QS4ML-VU-100\final_data


In [3]:
#此处所有文件都是raw文件

heart_file_path = 'band_data\\raw\\'
heart_rate_file = 'heart-rate-raw.csv'

band_file_path = 'band_data\\raw\\'
band_file=['band-walk-raw.csv','band-bike-raw.csv','band-run-raw.csv','band-sit-raw.csv','band-syn-raw.csv']

mat_file_path = 'mat_data\\raw\\'
mat_file =['sensorlog_20240606_114748 walk-raw.csv','sensorlog_20240606_115847 cycling-raw.csv',
           'sensorlog_20240606_120745 run-raw.csv','sensorlog_20240606_121202 sit-raw.csv','sensorlog_20240606_121611 synthesis-raw.csv']

phone_file_path = 'phone_data\\raw\\'
phone_file = ['test_phone.csv','train_phone.csv']



In [4]:
#处理phone数据
df_phone_test = pd.read_csv(os.getcwd().replace('final_data',phone_file_path)+ phone_file[0])
df_phone_train = pd.read_csv(os.getcwd().replace('final_data',phone_file_path)+ phone_file[1])
#合并两个表
df_phone = pd.concat([df_phone_test,df_phone_train],ignore_index=True)
# 重命名时间戳，统一为dateTime
df_phone.rename(columns={'time': 'dateTime'}, inplace=True)
df_phone.rename(columns={'type': 'deviceStatus'}, inplace=True)
# 把时间戳转化为datetime格式
df_phone['dateTime'] = pd.to_datetime(df_phone['dateTime'])
# 按时间戳排序
df_phone = df_phone.sort_values(by='dateTime')
# 重置索引
df_phone = df_phone.reset_index(drop=True)

#处理heart数据
df_heart_rate = pd.read_csv(os.getcwd().replace('final_data',heart_file_path)+heart_rate_file)

df_heart_rate['dateTime'] = pd.to_datetime(df_heart_rate['dateTime'])


In [11]:
#合并后数据的存储路径
raw_marge_file=['walk_merge_raw.csv','bike_merge_raw.csv','run_merge_raw.csv','sit_merge_raw.csv','syn_merge_raw.csv']

for i in range(0,5):
    #读取band数据
    df_band=pd.read_csv(os.getcwd().replace('final_data',band_file_path)+ band_file[i])
    df_band['dateTime'] = pd.to_datetime(df_band['dateTime'])
    #读取mat数据
    df_mat=pd.read_csv(os.getcwd().replace('final_data',mat_file_path)+ mat_file[i])
    df_mat.rename(columns={'Timestamp': 'dateTime'}, inplace=True)
    df_mat['dateTime'] = pd.to_datetime(df_mat['dateTime'])
    
    
    #计算时间起始
    band_start = df_band['dateTime'][0]
    band_end = df_band['dateTime'][len(df_band)-1]
    mat_start = df_mat['dateTime'][0]
    mat_end = df_mat['dateTime'][len(df_mat)-1]
    #计算合并时间起始
    merge_time_start = min(band_start,mat_start)
    merge_time_end = max(band_end,mat_end)
    
    #筛选心跳和手机数据的时间段
    heart_temp = df_heart_rate[(df_heart_rate['dateTime']>=merge_time_start) & (df_heart_rate['dateTime']<=merge_time_end)]
    phone_temp = df_phone[(df_phone['dateTime']>=merge_time_start) & (df_phone['dateTime']<=merge_time_end)]
    
    dfs=[df_band,df_mat,heart_temp,phone_temp]
    #合并数据
    merge_result = reduce(lambda left, right: pd.merge(left, right, on='dateTime', how='outer'), dfs)
    #筛选合并后的数据时间段
    merge_result = merge_result[(merge_result['dateTime']>=merge_time_start) & (merge_result['dateTime']<=merge_time_end)]
    #保存合并后的数据
    merge_result.to_csv(os.getcwd()+'//raw_merge_data//'+raw_marge_file[i],index=False)
      